In [ ]:
import pandas as pd
import numpy as np
from pycaret.classification import *
import joblib
import os
from datetime import datetime


In [ ]:
def load_data(file_path):
    """
    Carga los datos del archivo CSV con manejo de encoding
    
    Args:
        file_path (str): Ruta al archivo CSV
        
    Returns:
        pd.DataFrame: DataFrame con los datos cargados
    """
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        print(f"Datos cargados exitosamente con encoding UTF-8")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
        print(f"Datos cargados exitosamente con encoding ISO-8859-1")
    
    print(f"Forma del dataset: {df.shape}")
    return df

In [ ]:
df = load_data("../data/retail_features_dataset.csv")

In [ ]:
def setup_pycaret_environment(df, target_column='WillPurchase_90Days', test_size=0.2):
    """
    Configura el entorno de PyCaret para clasificación
    
    Args:
        df (pd.DataFrame): DataFrame con los datos
        target_column (str): Nombre de la columna objetivo
        test_size (float): Proporción del conjunto de prueba (default: 0.2)
        
    Returns:
        object: Objeto de configuración de PyCaret
    """
    print("Configurando entorno de PyCaret...")
    
    # Configurar PyCaret
    clf = setup(
        data=df,
        target=target_column,
        train_size=1-test_size,
        session_id=123,
        use_gpu=True
    )
    
    print(f"Dataset dividido: {int((1-test_size)*100)}% entrenamiento, {int(test_size*100)}% prueba")
    print("Configuración completada exitosamente")
    
    return clf

def compare_models_and_select_best(top_n=10):
    """
    Compara múltiples algoritmos de machine learning y selecciona los mejores
    
    Args:
        top_n (int): Número de mejores modelos a mostrar (default: 10)
        
    Returns:
        pd.DataFrame: DataFrame con la comparación de modelos
    """
    print(f"Comparando algoritmos de machine learning (Top {top_n})...")
    
    # Comparar modelos
    best_models = compare_models(
        sort='Accuracy',
        n_select=top_n,
        verbose=False
    )
    
    print("Comparación de modelos completada")
    return best_models

def create_and_tune_best_model(model_list, metric='Accuracy'):
    """
    Crea y optimiza el mejor modelo de la lista
    
    Args:
        model_list: Lista de modelos de PyCaret
        metric (str): Métrica para seleccionar el mejor modelo
        
    Returns:
        object: Modelo optimizado
    """
    print("Seleccionando y optimizando el mejor modelo...")
    
    # Seleccionar el mejor modelo (el primero de la lista ordenada)
    best_model = model_list[0] if isinstance(model_list, list) else model_list
    
    print(f"Modelo seleccionado: {type(best_model).__name__}")
    
    # Optimizar hiperparámetros
    print("Optimizando hiperparámetros...")
    tuned_model = tune_model(
        best_model,
        optimize=metric,
        verbose=False
    )
    
    print("Optimización completada")
    return tuned_model

def evaluate_model_performance(model):
    """
    Evalúa el rendimiento del modelo
    
    Args:
        model: Modelo entrenado de PyCaret
        
    Returns:
        dict: Diccionario con métricas de evaluación
    """
    print("Evaluando rendimiento del modelo...")
    
    # Evaluar modelo
    evaluation = evaluate_model(model)
    
    # Obtener métricas
    predictions = predict_model(model, verbose=False)
    
    print("Evaluación completada")
    return evaluation, predictions

def finalize_and_save_model(model, model_name="best_purchase_prediction_model"):
    """
    Finaliza el modelo y lo guarda en formato .pkl
    
    Args:
        model: Modelo optimizado
        model_name (str): Nombre base para el archivo del modelo
        
    Returns:
        str: Ruta del archivo guardado
    """
    print("Finalizando modelo...")
    
    # Finalizar modelo (entrena con todo el dataset)
    final_model = finalize_model(model)
    
    # Crear directorio para modelos si no existe
    models_dir = "../models"
    os.makedirs(models_dir, exist_ok=True)
    
    # Generar nombre de archivo con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{model_name}_{timestamp}"
    filepath = os.path.join(models_dir, filename)
    
    # Guardar modelo
    saved_model = save_model(final_model, filepath, verbose=False)
    
    pkl_path = f"{filepath}.pkl"
    print(f"Modelo guardado exitosamente en: {pkl_path}")
    
    return pkl_path, final_model

def get_feature_importance(model):
    """
    Obtiene la importancia de las características del modelo
    
    Args:
        model: Modelo entrenado
        
    Returns:
        pd.DataFrame: DataFrame con la importancia de características
    """
    try:
        print("Calculando importancia de características...")
        
        # Intentar obtener importancia de características
        importance_plot = plot_model(model, plot='feature', verbose=False, display_format='dataframe')
        
        print("Importancia de características calculada")
        return importance_plot
    except Exception as e:
        print(f"No se pudo calcular la importancia de características: {str(e)}")
        return None

def complete_modeling_pipeline(df, target_column='WillPurchase_90Days'):
    """
    Pipeline completo de modelado con PyCaret
    
    Args:
        df (pd.DataFrame): DataFrame con los datos
        target_column (str): Columna objetivo
        
    Returns:
        tuple: (modelo_final, ruta_del_archivo, métricas_evaluación)
    """
    print("Iniciando pipeline completo de modelado...")
    
    # 1. Configurar entorno
    setup_result = setup_pycaret_environment(df, target_column)
    
    # 2. Comparar modelos
    best_models = compare_models_and_select_best(top_n=5)
    
    # 3. Optimizar mejor modelo
    tuned_model = create_and_tune_best_model(best_models)
    
    # 4. Evaluar modelo
    evaluation, predictions = evaluate_model_performance(tuned_model)
    
    # 5. Obtener importancia de características
    feature_importance = get_feature_importance(tuned_model)
    
    # 6. Finalizar y guardar modelo
    model_path, final_model = finalize_and_save_model(tuned_model)
    
    print("Pipeline de modelado completado exitosamente")
    
    return final_model, model_path, evaluation, predictions, feature_importance

In [ ]:
def load_model_and_predict(model_path, new_data):
    """
    Carga un modelo guardado y realiza predicciones
    
    Args:
        model_path (str): Ruta al archivo .pkl del modelo
        new_data (pd.DataFrame): Datos para hacer predicciones
        
    Returns:
        pd.DataFrame: DataFrame con predicciones
    """
    print(f"Cargando modelo desde: {model_path}")
    
    # Cargar modelo
    loaded_model = load_model(model_path.replace('.pkl', ''))
    
    # Realizar predicciones
    predictions = predict_model(loaded_model, data=new_data, verbose=False)

    print("Predicciones realizadas exitosamente")
    return predictions

In [ ]:
def predict_new_customers(model_path, customer_data):
    """
    Predice si nuevos clientes comprarán en los próximos 90 días
    
    Args:
        model_path (str): Ruta al modelo guardado
        customer_data (pd.DataFrame): Datos de los clientes
        
    Returns:
        pd.DataFrame: Predicciones con probabilidades
    """
    print("Realizando predicciones para nuevos clientes...")
    
    predictions = load_model_and_predict(model_path, customer_data)
    
    
    # Agregar interpretación de resultados
    predictions['Prediction'] = predictions['prediction_label'].map({
        1: 'Buy in 90 days',
        0: 'Will not buy in 90 days'
    })
    
    print(f"Predicciones completadas para {len(predictions)} clientes")
    
    return predictions

In [ ]:
final_model, model_path, evaluation, predictions, feature_importance = complete_modeling_pipeline(df)

# Mostrar resultados
print("\nResumen del modelo:")
print(f"Modelo guardado en: {model_path}")
print(f"Tipo de modelo: {type(final_model).__name__}")

In [ ]:
new_customer_data = df.drop('WillPurchase_90Days', axis=1).sample(10)

In [ ]:
predictions = predict_new_customers(
    model_path="../models/best_purchase_prediction_model_20250724_200929.pkl",
    customer_data=new_customer_data,
    )

In [ ]:
predictions

In [ ]:
predictions.to_csv("../data/predictions.csv")